In [6]:
#Importations
from PIL import Image
import numpy as np 

[255   0   0]


In [7]:
## TEST 0: Generating an image of a given size and saving it. 

w, h = 32, 32

data = np.zeros((h, w, 3), dtype = np.uint8)
data[0,0] = [255, 0,0]

print(data[0,0])

img = Image.fromarray(data, 'RGB')
img.save('images/produced/test0.png')

[255   0   0]


In [18]:
## TEST 1: Heatmap generation with a given function with known bounds. 0<F(x)<M // F: R^2 -> R

def f1(x,c):
    ## x; c: two dimensional
    # c: center of appliaction.
    z = x - c
    return np.sin( (z[0]*z[1])*0.05)**2

w,h = 256, 256
data = np.zeros((h,w,3), dtype = np.uint8)

c = np.array([h/2, w/2])

for i in range(w):
    for j in range(h):
        data[i,j] = [ f1( np.array([i,j]), c )*255, 0, 0]
img = Image.fromarray(data, 'RGB')
img.save('images/produced/test1.png')


In [30]:
## TEST 2: Automatic rescaling of heat

def heat_rescaling(A):
    '''
    A: 2 dim array.
    Find the max and minimal value of M and gives a rescaling on [0,255].
    '''

    # ABSTRACT
    # note m and M the minimal and maximal values of the array.
    # we want to find the coefficients a and b st:
    # a*m + b = 0
    # a*M + b = 255
    #
    # => a(M-m)= 255 so a = 255/(M-m)
    # m*255/(M-m) = -b so b = m*255/(m-M)

    m = A.min()
    M = A.max()
    if m == M:
        print("Error: all values of the matrix are identical.")
        return None
    return 255/(M-m), m*255/(m-M)


#A = np.array([[-1,3], [0,0]])
#heat_rescaling(A)

## APPLICATION: given any f: R2 -> R do the 1 dimensionnal heatmap.

def f2(x,c):
    z = x-c
    return np.sin(z[0]*0.05) + np.cos(z[1]*0.1) ## here we have vals in [0,1]

w,h = 256, 256
data = np.zeros((h,w, 3), dtype = np.uint8) #/!\ IS 3D
TMP = np.zeros((h,w), dtype = np.float32) # /!\ IS 2D

#f2 application on the 2-dim array (as heat_rescaling works on 2-d array!)
c= np.array([h/2, w/2])
for i in range(w):
    for j in range(h):
        TMP[i,j] = f2([i,j], c)

#find the rescaling parameters
a,b = heat_rescaling(TMP)

#application in the data array of rescaled values
for i in range(w):
    for j in range(h):
        data[i,j] = [a*TMP[i,j]+b, 0, 0]

img = Image.fromarray(data, 'RGB')
img.save('images/produced/test2.png')


In [43]:
## TEST 3: multiple color heat brush
#TODO: change the DPI functions so that DPI_B and DPI_R are more present, and DPI_G less. 
#
# idea: instead of having black & red images, use the whole RGB palette.
#
# for instance: 
#
#
# 
#      BV   GV     RV       |
#      /\    /\    /\       | Designated 
#     /  \  /  \  /  \      | Palette
#    /    \/    \/    \     | Intensity
#   /     /\    /\     \    |
#   ----------------------  | I love graphic design
#       |0          |255 
# Init.Intensity (0-255)
# 
# how to do it: 
# to avoid having very red and very black pixels on the same image we should ensure that for any point, the sum of DPI is always 255. 
# we can use hat functions to do so.
#
# DPI functions return, given an initial intensity, a value for the designated color.
# thus we shall have somewhere in the code smthg like:
#
#   val = TMP[i,j]
#   data = DPI(val)
#
# with
#   DPI(val) is a 3 value array.

def DPI1_B(val):
    return max(-255*val/128 + 255, 0)

def DPI1_R(val):
    return max(0, 255*val/128- 255)

def DPI1(val):
    bv = DPI1_B(val)
    rv = DPI1_R(val)
    gv = 255 -bv -rv
    return [rv, gv, bv]

def f3(x,c):
    z = x-c
    return np.linalg.norm(z, 2)

w,h = 128, 128
data = np.zeros((h,w, 3), dtype = np.uint8) #/!\ IS 3D
TMP = np.zeros((h,w), dtype = np.float32) # /!\ IS 2D

#f2 application on the 2-dim array (as heat_rescaling works on 2-d array!)
c= np.array([h/2, w/2])
for i in range(w):
    for j in range(h):
        TMP[i,j] = f3([i,j], c)

#find the rescaling parameters
a,b = heat_rescaling(TMP)

#application in the data array of rescaled values
for i in range(w):
    for j in range(h):
        val = a*TMP[i,j]+b
        data[i,j] = DPI1(val)

img = Image.fromarray(data, 'RGB')
img.save('images/produced/test3.png')